In [ ]:
import json
from notebookutils import mssparkutils
import pandas as pd
from pyspark.sql import Row
Source = 'abfss://datalakelanding@arkstgdlsadsaqyeadsl.dfs.core.windows.net/accounts-response.json'
sourceDF = spark.read.option("multiline","true").json(Source)
sourceDF.show()
#used for outputting to json in next cell block
Target = 'abfss://datalakelanding@arkstgdlsadsaqyeadsl.dfs.core.windows.net/test-response.json'


In [ ]:
#flattening of json stored in a dataframe (elements column, first row)
#using function

#This function will take a dictionary and build it out completely for columns in a dataframe (so a nested dictionary / json )
def as_row(obj):
    if isinstance(obj, dict):
        dictionary = {k: as_row(v) for k, v in obj.items()}
        return Row(**dictionary)
    elif isinstance(obj, list):
        return [as_row(v) for v in obj]
    else:
        return obj

#collect each array object of 'elements' and convert it to a list of dictionaries
ss=sourceDF.select("elements").collect()
output=[i[0] for i in ss]
fullList = []
for obj in output:
    for row in obj:
        fullList.append(row.asDict(True))

#we now want to iterate this list of dictionaries (or list of jsons) for manipulation and re-building into an amended list
amendedList = []
for obj in fullList:
    obj["id"] = str(obj["id"])
    row = as_row(obj)
    amendedList.append(row)


#for testing
#for x in amendedList:
#    print(type(x))
outDF = spark.createDataFrame(amendedList)
outDF.show()

#this was for testing writing to json 
#newDF.write.format("json").mode("overwrite").save(Target)


In [ ]:
#taking a json array in dataframe (elements column, first row) and making it multiple rows of json objects in a new dataframe

#collect each array object of 'elements' and convert it to a list of dictionaries
ss=sourceDF.select("elements").collect()
output=[i[0] for i in ss]
fullList = []
for obj in output:
    for row in obj:
        fullList.append(row.asDict(True))

#we now want to iterate this list of dictionaries (or list of jsons) for manipulation and re-building into an amended list
amendedList = []
for obj in fullList:
    obj["id"] = str(obj["id"])
    amendedList.append(obj)


#re-build - note: pyspark is a bit funky currently with the first run using pandas, will throw a warning and hide output, but normally still executes full code block (check for successful run)
pdDF = pd.DataFrame([str(e) for e in amendedList])
columns = ["elements"]
outDF = spark.createDataFrame(pdDF, columns)
outDF.show()

#this was for testing writing to json 
#newDF.write.format("json").mode("overwrite").save(Target)


In [99]:
#output = pd.DataFrame()
#for x in amendedList:
#    output = output.append(x, ignore_index=True)
#sparkDF=spark.createDataFrame(output)
#sparkDF.show() 
#for a in amendedList:
  #  print(e)


#df = pd.DataFrame(amendedList, columns=['elements'])
#df = spark.createDataFrame(df)
#df.show()


In [ ]:
"""
{
    "sink": {
        "storeSettings": {
            "type": "AzureBlobFSWriteSettings"
        },
        "type": "ParquetSink"
    },
    "mappings": [
        {
            "source": {
                "name": "LAST_NAME",
                "type": "String",
                "physicalType": "VARCHAR2"
            },
            "sink": {
                "name": "LASTNAME",
                "type": "UTF8",
                "physicalType": "VARCHAR2"
            }
        },
        {
            "source": {
                "name": "UNIQUE_EMAIL",
                "type": "String",
                "physicalType": "VARCHAR2"
            },
            "sink": {
                "name": "UNIQUEEMAIL",
                "type": "UTF8",
                "physicalType": "VARCHAR2"
            }
        },
        {
            "source": {
                "name": "FIRST_NAME",
                "type": "String",
                "physicalType": "VARCHAR2"
            },
            "sink": {
                "name": "FIRSTNAME",
                "type": "UTF8",
                "physicalType": "VARCHAR2"
            }
        },
        {
            "source": {
                "name": "PERSON_ID",
                "type": "Int64",
                "physicalType": "NUMBER"
            },
            "sink": {
                "name": "PERSONID",
                "type": "Int64",
                "physicalType": "NUMBER"
            }
        }
    ]
    
}
"""